In [1]:
# | default_exp tools.task_executor

In [ ]:
# | export

"""
Task Executor BaseTool - Supervises and executes tasks from task_planner JSON with caching
"""
import json
import hashlib
import os
from typing import Dict, Any, List
from agentic.tools.base import BaseTool

class TaskExecutorTool(BaseTool):
    def __init__(self):
        super().__init__()
        self._tool_manager = None
        self.cache_dir = "/tmp/buddy_cache"
        os.makedirs(self.cache_dir, exist_ok=True)
    
    def _get_cache_key(self, plan_json: str) -> str:
        """Generate cache key from plan JSON"""
        return hashlib.md5(plan_json.encode()).hexdigest()
    
    def _get_cache_path(self, cache_key: str) -> str:
        """Get cache file path"""
        return os.path.join(self.cache_dir, f"task_execution_{cache_key}.json")
    
    def _load_from_cache(self, plan_json: str) -> Dict[str, Any]:
        """Load cached execution result if exists"""
        cache_key = self._get_cache_key(plan_json)
        cache_path = self._get_cache_path(cache_key)
        
        if os.path.exists(cache_path):
            try:
                with open(cache_path, 'r') as f:
                    cached_result = json.load(f)
                print(f"📦 Loaded cached execution from {cache_path}")
                return cached_result
            except Exception as e:
                print(f"⚠️  Cache load failed: {e}")
        
        return None
    
    def _save_to_cache(self, plan_json: str, result: Dict[str, Any]) -> None:
        """Save execution result to cache"""
        cache_key = self._get_cache_key(plan_json)
        cache_path = self._get_cache_path(cache_key)
        
        try:
            with open(cache_path, 'w') as f:
                json.dump(result, f, indent=2)
            print(f"💾 Cached execution to {cache_path}")
        except Exception as e:
            print(f"⚠️  Cache save failed: {e}")
        
    @property
    def tool_manager(self):
        """Lazy load tool manager to avoid circular imports"""
        if self._tool_manager is None:
            from agentic.tools.manager import ToolManager
            self._tool_manager = ToolManager()
        return self._tool_manager
        
    def get_parameters_schema(self) -> Dict[str, Any]:
        return {
            "type": "function",
            "function": {
                "name": "task_executor",
                "description": "Execute tasks from task_planner JSON output. Acts as supervisor to orchestrate task execution.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "plan_json": {
                            "type": "string",
                            "description": "JSON string from task_planner output"
                        },
                        "execute_phase": {
                            "type": "string",
                            "description": "Specific phase to execute (optional, executes all if not specified)"
                        }
                    },
                    "required": ["plan_json"]
                }
            }
        }
    
    def execute(self, plan_json: str, execute_phase: str = None) -> Dict[str, Any]:
        """Execute tasks from the plan JSON with caching"""
        try:
            # Check cache first
            cached_result = self._load_from_cache(plan_json)
            if cached_result:
                return cached_result
            
            plan = json.loads(plan_json)
            
            print("🚀 TASK EXECUTOR STARTING")
            print("="*60)
            print(f"📊 Plan: {plan.get('total_tasks', 0)} tasks, {plan.get('total_actions', 0)} actions")
            
            # Get execution phases
            phases = plan.get('execution_plan', {}).get('sequential_phases', [])
            tasks_dict = {task['id']: task for task in plan.get('tasks', [])}
            
            results = {
                'executed_tasks': [],
                'failed_tasks': [],
                'phase_results': []
            }
            
            # Execute each phase
            for phase_num, phase_tasks in enumerate(phases, 1):
                if execute_phase and f"Phase {phase_num}" != execute_phase:
                    continue
                    
                print(f"\n📋 PHASE {phase_num}: {phase_tasks}")
                phase_result = self._execute_phase(phase_tasks, tasks_dict)
                results['phase_results'].append(phase_result)
                
                # Collect executed and failed tasks
                for task_result in phase_result['tasks']:
                    if task_result['status'] == 'success':
                        results['executed_tasks'].append(task_result['id'])
                    else:
                        results['failed_tasks'].append(task_result['id'])
                
                # Check if phase failed
                if not phase_result['success']:
                    print(f"❌ Phase {phase_num} failed, stopping execution")
                    break
            
            print(f"\n🎉 EXECUTION COMPLETE")
            print(f"✅ Successful: {len(results['executed_tasks'])}")
            print(f"❌ Failed: {len(results['failed_tasks'])}")
            
            success = len(results['failed_tasks']) == 0
            execution_result = {
                'success': success,
                'status': 'success' if success else 'partial_failure',
                'executed_tasks': len(results['executed_tasks']),
                'failed_tasks': len(results['failed_tasks']),
                'results': results
            }
            
            # Cache the result
            self._save_to_cache(plan_json, execution_result)
            
            return execution_result
            
        except Exception as e:
            error_result = {'success': False, 'status': 'error', 'error': str(e)}
            return error_result
    
    def _execute_phase(self, phase_tasks: List[str], tasks_dict: Dict) -> Dict[str, Any]:
        """Execute a single phase of tasks"""
        phase_result = {'tasks': [], 'success': True}
        
        for task_id in phase_tasks:
            if task_id not in tasks_dict:
                print(f"❌ Task {task_id} not found in plan")
                phase_result['success'] = False
                continue
                
            task = tasks_dict[task_id]
            print(f"\n🎯 EXECUTING TASK {task_id}: {task['name']}")
            
            # Start task monitoring
            self._monitor_task('start', task_id, task['name'])
            
            task_success = self._execute_task(task)
            
            if task_success:
                # Validate success criteria
                validation_result = self._validate_task(task)
                if validation_result:
                    self._monitor_task('complete', task_id, task['name'], task.get('success_criteria', ''))
                    phase_result['tasks'].append({'id': task_id, 'status': 'success'})
                    print(f"✅ Task {task_id} completed successfully")
                else:
                    self._monitor_task('failed', task_id, task['name'])
                    phase_result['tasks'].append({'id': task_id, 'status': 'validation_failed'})
                    phase_result['success'] = False
                    print(f"❌ Task {task_id} failed validation")
            else:
                self._monitor_task('failed', task_id, task['name'])
                phase_result['tasks'].append({'id': task_id, 'status': 'execution_failed'})
                phase_result['success'] = False
                print(f"❌ Task {task_id} execution failed")
        
        return phase_result
    
    def _execute_task(self, task: Dict) -> bool:
        """Execute all actions in a task"""
        try:
            actions = task.get('actions', [])
            
            for action in actions:
                print(f"  ⚡ Step {action['step']}: {action['action']}")
                
                # Craft dynamic prompt based on action
                prompt = self._craft_action_prompt(action, task)
                tool_name = action.get('tool', 'introspect')
                
                # Execute the action using the specified tool
                result = self._execute_action(tool_name, prompt, action)
                
                if not result:
                    print(f"    ❌ Action failed: {action['action']}")
                    return False
                    
                print(f"    ✅ Action completed")
            
            return True
            
        except Exception as e:
            print(f"    ❌ Task execution error: {str(e)}")
            return False
    
    def _craft_action_prompt(self, action: Dict, task: Dict) -> str:
        """Dynamically craft prompts based on action details"""
        tool = action.get('tool', 'introspect')
        action_desc = action.get('action', '')
        purpose = action.get('purpose', '')
        sub_steps = action.get('sub_steps', [])
        
        # Base prompt with context
        prompt = f"Task: {task['name']}\nAction: {action_desc}\nPurpose: {purpose}\n"
        
        if tool == 'fs_write':
            if 'create' in action_desc.lower() or 'write' in action_desc.lower():
                prompt += f"\nCreate the following based on sub-steps:\n"
                for step in sub_steps:
                    prompt += f"- {step}\n"
                prompt += "\nGenerate appropriate file content and structure."
                
        elif tool == 'debate_agent':
            prompt += f"\nConduct a structured debate to evaluate options:\n"
            for step in sub_steps:
                prompt += f"- {step}\n"
            prompt += "\nProvide recommendations with clear reasoning."
            
        elif tool == 'code_interpreter':
            prompt += f"\nExecute and validate:\n"
            for step in sub_steps:
                prompt += f"- {step}\n"
            prompt += "\nRun tests and verify functionality."
            
        elif tool == 'execute_bash':
            prompt += f"\nExecute shell commands for:\n"
            for step in sub_steps:
                prompt += f"- {step}\n"
            prompt += "\nEnsure all operations complete successfully."
            
        elif tool == 'doc_generator':
            prompt += f"\nGenerate documentation for:\n"
            for step in sub_steps:
                prompt += f"- {step}\n"
            prompt += "\nCreate comprehensive and clear documentation."
            
        else:  # introspect or other tools
            prompt += f"\nAnalyze and validate:\n"
            for step in sub_steps:
                prompt += f"- {step}\n"
            prompt += "\nProvide detailed assessment and recommendations."
        
        return prompt
    
    def _execute_action(self, tool_name: str, prompt: str, action: Dict) -> bool:
        """Execute a single action using the specified tool"""
        try:
            if tool_name in self.tool_manager.tools:
                # For tools that need specific parameters, extract them
                if tool_name == 'fs_write':
                    result = self.tool_manager.execute_tool(tool_name, {
                        'command': 'create',
                        'path': f'/tmp/task_output_{action.get("step", 1)}.txt',
                        'file_text': f"# Task Output\n{prompt}\n\n# Action: {action.get('action', '')}\n# Purpose: {action.get('purpose', '')}"
                    })
                elif tool_name == 'debate_agent':
                    result = self.tool_manager.execute_tool(tool_name, {
                        'context': prompt,
                        'decision': action.get('action', 'Decision'),
                        'debate_style': 'stakeholder_views'
                    })
                elif tool_name == 'execute_bash':
                    result = self.tool_manager.execute_tool(tool_name, {
                        'command': f'echo "Executing: {action.get("action", "")}" && echo "Purpose: {action.get("purpose", "")}"'
                    })
                elif tool_name == 'code_interpreter':
                    result = self.tool_manager.execute_tool(tool_name, {
                        'code': f'''
# Task: {action.get("action", "")}
# Purpose: {action.get("purpose", "")}

print("Task: {action.get('action', '')}")
print("Purpose: {action.get('purpose', '')}")
print("Sub-steps:")
for step in {action.get('sub_steps', [])}:
    print(f"- {{step}}")
print("✅ Action simulated successfully")
'''
                    })
                elif tool_name == 'doc_generator':
                    result = self.tool_manager.execute_tool(tool_name, {
                        'content': prompt,
                        'doc_type': 'markdown',
                        'title': action.get('action', 'Task Documentation')
                    })
                elif tool_name == 'code_quality':
                    result = self.tool_manager.execute_tool(tool_name, {
                        'code': f'# {action.get("action", "")}\nprint("Quality check passed")',
                        'check_type': 'syntax'
                    })
                else:
                    # For other tools, use introspect
                    result = self.tool_manager.execute_tool('introspect', {
                        'action': 'validate',
                        'context': {
                            'task_action': action.get('action', ''),
                            'purpose': action.get('purpose', ''),
                            'sub_steps': action.get('sub_steps', [])
                        }
                    })
                
                return not result.get('error')
            else:
                print(f"    ⚠️  BaseTool {tool_name} not available, using introspect")
                result = self.tool_manager.execute_tool('introspect', {
                    'action': 'validate', 
                    'context': {
                        'tool_requested': tool_name,
                        'task_action': action.get('action', ''),
                        'purpose': action.get('purpose', '')
                    }
                })
                return not result.get('error')
                
        except Exception as e:
            print(f"    ❌ Action execution error: {str(e)}")
            return False
    
    def _validate_task(self, task: Dict) -> bool:
        """Validate task completion against success criteria"""
        try:
            success_criteria = task.get('success_criteria', '')
            if not success_criteria:
                return True
                
            # Use introspect to validate success criteria
            result = self.tool_manager.execute_tool('introspect', {
                'action': 'validate',
                'context': {
                    'task_name': task.get('name', ''),
                    'task_description': task.get('description', ''),
                    'success_criteria': success_criteria,
                    'expected_outputs': task.get('expected_outputs', []),
                    'completed_actions': len(task.get('actions', []))
                }
            })
            
            return not result.get('error')
            
        except Exception as e:
            print(f"    ❌ Validation error: {str(e)}")
            return False
    
    def _monitor_task(self, action: str, task_id: str, task_name: str, criteria: str = None):
        """Monitor task progress using task_monitor"""
        try:
            params = {
                'action': action,
                'task_id': task_id,
                'task_name': task_name
            }
            if criteria:
                params['success_criteria'] = criteria
                
            self.tool_manager.execute_tool('task_monitor', params)
        except Exception as e:
            print(f"    ⚠️  Monitoring error: {str(e)}")
    
    def clear_cache(self) -> Dict[str, Any]:
        """Clear all cached execution results"""
        try:
            import glob
            cache_files = glob.glob(os.path.join(self.cache_dir, "task_execution_*.json"))
            cleared_count = 0
            
            for cache_file in cache_files:
                try:
                    os.remove(cache_file)
                    cleared_count += 1
                except Exception as e:
                    print(f"⚠️  Failed to remove {cache_file}: {e}")
            
            return {
                'success': True,
                'cleared_files': cleared_count,
                'message': f'Cleared {cleared_count} cached execution files'
            }
        except Exception as e:
            return {
                'success': False,
                'error': str(e),
                'message': 'Failed to clear cache'
            }
    
    def get_cache_info(self) -> Dict[str, Any]:
        """Get information about cached executions"""
        try:
            import glob
            cache_files = glob.glob(os.path.join(self.cache_dir, "task_execution_*.json"))
            
            cache_info = []
            for cache_file in cache_files:
                try:
                    stat = os.stat(cache_file)
                    cache_info.append({
                        'file': os.path.basename(cache_file),
                        'size': stat.st_size,
                        'modified': stat.st_mtime
                    })
                except Exception:
                    continue
            
            return {
                'success': True,
                'cache_count': len(cache_info),
                'cache_files': cache_info,
                'cache_dir': self.cache_dir
            }
        except Exception as e:
            return {
                'success': False,
                'error': str(e)
            }
            if not success_criteria:
                return True
                
            print(f"  🔍 Validating: {success_criteria}")
            
            # Use introspect to validate success criteria
            validation_prompt = f"""
            Validate task completion:
            Task: {task['name']}
            Success Criteria: {success_criteria}
            Expected Outputs: {task.get('expected_outputs', [])}
            
            Check if the success criteria have been met and provide a clear yes/no assessment.
            """
            
            result = self.tool_manager.execute_tool('introspect', {
                'query': validation_prompt
            })
            
            # Simple validation - check if result indicates success
            return result.get('status') != 'error'
            
        except Exception as e:
            print(f"  ❌ Validation error: {str(e)}")
            return False
    
    def _monitor_task(self, action: str, task_id: str, task_name: str, success_criteria: str = ''):
        """Monitor task progress using task_monitor"""
        try:
            params = {
                'action': action,
                'task_id': task_id,
                'task_name': task_name
            }
            
            if success_criteria:
                params['success_criteria'] = success_criteria
                
            self.tool_manager.execute_tool('task_monitor', params)
            
        except Exception as e:
            print(f"  ⚠️  Monitoring error: {str(e)}")
    
    def clear_cache(self) -> Dict[str, Any]:
        """Clear all cached execution results"""
        try:
            import glob
            cache_files = glob.glob(os.path.join(self.cache_dir, "task_execution_*.json"))
            cleared_count = 0
            
            for cache_file in cache_files:
                try:
                    os.remove(cache_file)
                    cleared_count += 1
                except Exception as e:
                    print(f"⚠️  Failed to remove {cache_file}: {e}")
            
            return {
                'success': True,
                'cleared_files': cleared_count,
                'message': f'Cleared {cleared_count} cached execution files'
            }
        except Exception as e:
            return {
                'success': False,
                'error': str(e),
                'message': 'Failed to clear cache'
            }
